In [12]:
import pandas as pd
import requests
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine, text

In [13]:
load_dotenv()

True

In [14]:
json_data = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').json()

In [15]:
df = pd.DataFrame(json_data["events"])

In [16]:
df = df[["name", "deadline_time", "average_entry_score", "highest_scoring_entry","deadline_time_epoch","highest_score","ranked_count", "finished"]]

In [17]:
df.head()

,name,deadline_time,average_entry_score,highest_scoring_entry,deadline_time_epoch,highest_score,ranked_count,finished
0,Gameweek 1,2025-08-15T17:30:00Z,54,3772644.0,1755279000,127.0,9469118,True
1,Gameweek 2,2025-08-22T17:30:00Z,51,2963810.0,1755883800,140.0,10752422,True
2,Gameweek 3,2025-08-30T10:00:00Z,48,4374360.0,1756548000,118.0,11304791,True
3,Gameweek 4,2025-09-13T10:00:00Z,63,7997042.0,1757757600,139.0,11641518,True
4,Gameweek 5,2025-09-20T10:00:00Z,42,9985225.0,1758362400,112.0,11803614,True


In [18]:
engine = create_engine(os.getenv("POSTGRES_URI") or "")


In [19]:
with engine.begin() as con:
    con.execute(
        text(
            '''
            create table if not exists stg_events(
                event_id int generated always as identity primary key not null,
                name varchar(100) not null,
                deadline_time date ,
                average_entry_score int,
                highest_scoring_entry int,
                deadline_time_epoch int,
                highest_score int,
                ranked_count int,
                finished bool
            )
            '''
        )
    )
    
    con.execute(
        text(
            "truncate table stg_events restart identity"
        )
    )

In [20]:
df.to_sql("stg_events", con=engine, if_exists="append", index=False)

38

In [21]:
with engine.begin() as con:
    con.execute(
        text(
            '''
            MERGE INTO events AS target
            USING stg_events AS source
            ON target.event_id = source.event_id
            
            WHEN MATCHED THEN
                UPDATE SET 
                    name = source.name,
                    deadline_time = source.deadline_time,
                    average_entry_score = source.average_entry_score,
                    highest_scoring_entry = source.highest_scoring_entry,
                    deadline_time_epoch = source.deadline_time_epoch,
                    highest_score = source.highest_score,
                    ranked_count = source.ranked_count,
                    finished = source.finished
            
            WHEN NOT MATCHED THEN
                INSERT (name, deadline_time, average_entry_score, highest_scoring_entry, deadline_time_epoch, highest_score, ranked_count, finished)
                VALUES (source.name, source.deadline_time, source.average_entry_score, source.highest_scoring_entry, source.deadline_time_epoch, source.highest_score, source.ranked_count, source.finished);
            '''
        )
    )